In [117]:
import pandas as pd
import numpy as np
import ast


In [118]:
csv_name = 'video8'

df = pd.read_csv(f'data/{csv_name}.csv')
df.head(15)

,frame,shepherd,frame_bboxes,min_distance,max_distance,avg_distance,furthest_sheep_distance
0,0,robot,"[(1, 'sheep', [260, 628, 638, 862]), (2, 'shee...",NaN,NaN,NaN,1034.203558
1,1,robot,"[(1, 'sheep', [260, 629, 638, 862]), (2, 'shee...",NaN,NaN,NaN,1033.859758
2,2,robot,"[(1, 'sheep', [260, 629, 637, 862]), (2, 'shee...",NaN,NaN,NaN,1033.218515
3,3,robot,"[(1, 'sheep', [259, 629, 637, 862]), (2, 'shee...",NaN,NaN,NaN,1034.157870
4,4,robot,"[(1, 'sheep', [259, 629, 636, 861]), (2, 'shee...",NaN,NaN,NaN,1034.329372
5,5,robot,"[(1, 'sheep', [258, 629, 636, 861]), (2, 'shee...",NaN,NaN,NaN,1034.501087
6,6,robot,"[(1, 'sheep', [258, 629, 636, 861]), (2, 'shee...",NaN,NaN,NaN,1034.031552
7,7,robot,"[(1, 'sheep', [258, 629, 635, 861]), (2, 'shee...",NaN,NaN,NaN,1033.562045
8,8,robot,"[(1, 'sheep', [258, 629, 636, 860]), (2, 'shee...",NaN,NaN,NaN,1033.264729
9,9,robot,"[(1, 'sheep', [258, 629, 635, 860]), (2, 'shee...",NaN,NaN,NaN,1033.688178


In [119]:
df['frame_bboxes'] = df['frame_bboxes'].apply(ast.literal_eval)
df['frame_bboxes']

0        [(1, sheep, [260, 628, 638, 862]), (2, sheep, ...
1        [(1, sheep, [260, 629, 638, 862]), (2, sheep, ...
2        [(1, sheep, [260, 629, 637, 862]), (2, sheep, ...
3        [(1, sheep, [259, 629, 637, 862]), (2, sheep, ...
4        [(1, sheep, [259, 629, 636, 861]), (2, sheep, ...
                               ...                        
12620    [(917, sheep, [855, 429, 1021, 576]), (939, sh...
12621    [(917, sheep, [870, 425, 1024, 567]), (939, sh...
12622    [(917, sheep, [883, 420, 1026, 557]), (939, sh...
12623    [(917, sheep, [895, 410, 1032, 543]), (939, sh...
12624    [(917, sheep, [906, 402, 1033, 537]), (939, sh...
Name: frame_bboxes, Length: 12625, dtype: object

## Speed calculation

In [120]:
FPS = 30
frame_interval = 5

In [121]:
def calculate_distance(current_center, previous_center):
    return np.sqrt((current_center[0] - previous_center[0]) ** 2 + (current_center[1] - previous_center[1]) ** 2)


def calculate_speeds(df, frame_interval, fps):
    speeds = []
    dog_speeds = []
    
    for i in range(len(df)):
        if i < frame_interval:
            speeds.append([0] * len(df.loc[i, 'frame_bboxes']))
            dog_speeds.append(0)
            
        else:
            current_bboxes = df.loc[i, 'frame_bboxes']
            previous_bboxes = df.loc[i - frame_interval, 'frame_bboxes']

            frame_speeds = []
            dog_speed = 0
            for current_bbox in current_bboxes:
                object_id = current_bbox[0]
                current_center = [(current_bbox[2][0] + current_bbox[2][2]) / 2, (current_bbox[2][1] + current_bbox[2][3]) / 2] # Center of bbox
                previous_bbox = next((bbox for bbox in previous_bboxes if bbox[0] == object_id), None) # Gets the same box from previous frame by checking id

                if previous_bbox:
                    previous_center = [(previous_bbox[2][0] + previous_bbox[2][2]) / 2, (previous_bbox[2][1] + previous_bbox[2][3]) / 2] # Center of bbox
                    distance = calculate_distance(current_center, previous_center)
                    speed = distance / (frame_interval / fps)
                    frame_speeds.append((object_id, speed))
                    if current_bbox[1] == 'dog':  # Check if the object is a dog
                        dog_speed = speed
                else:
                    frame_speeds.append((object_id, 0))
                    if current_bbox[1] == 'dog':  # Check if the object is a dog
                        dog_speed = 0
                        
            speeds.append(frame_speeds)
            dog_speeds.append(dog_speed)
    
    return speeds, dog_speeds


In [122]:
def calculate_speeds(df, frame_interval, fps):
    speeds = []
    dog_speeds = []
    
    for i in range(len(df)):
        frame_speeds = []
        dog_speed = 0
        #print(f"ROW: {i}, len(dog_speeds):{len(dog_speeds)}")
        current_bboxes = df.loc[i, 'frame_bboxes']

        if current_bboxes:
            if i < frame_interval:
                for current_bbox in current_bboxes:
                    if current_bbox[1]!='dog':
                        frame_speeds.append((current_bbox[0], 0))
                dog_speed=0
                    
            else:
                previous_bboxes = df.loc[i - frame_interval, 'frame_bboxes']
            
                for current_bbox in current_bboxes:
                    object_id = current_bbox[0]
                    current_center = [(current_bbox[2][0] + current_bbox[2][2]) / 2, (current_bbox[2][1] + current_bbox[2][3]) / 2] # Center of bbox
                    previous_bbox = next((bbox for bbox in previous_bboxes if bbox[0] == object_id), None) # Gets the same box from previous frame by checking id

                    if previous_bbox:
                        previous_center = [(previous_bbox[2][0] + previous_bbox[2][2]) / 2, (previous_bbox[2][1] + previous_bbox[2][3]) / 2] # Center of bbox
                        distance = calculate_distance(current_center, previous_center)
                        speed = distance / (frame_interval / fps)


                        if current_bbox[1] == 'dog':  # Check if the object is a dog
                            dog_speed = speed
                        else: # Save as sheep speed
                            frame_speeds.append((object_id, speed))

                    else: # No matching detection
                        if current_bbox[1] == 'dog':  # Check if the object is a dog
                            dog_speed = 0
                        else:
                            frame_speeds.append((object_id, 0))

        speeds.append(frame_speeds)
        dog_speeds.append(dog_speed)
    
    return speeds, dog_speeds

In [123]:
df['speeds'], df['dog_speed'] = calculate_speeds(df, frame_interval-1, FPS)
df[['frame', 'speeds', 'dog_speed']].head(15)

,frame,speeds,dog_speed
0,0,"[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",0.0
1,1,"[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",0.0
2,2,"[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",0.0
3,3,"[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",0.0
4,4,"[(1, 11.25), (2, 0.0), (3, 5.303300858899107),...",0.0
5,5,"[(1, 15.461646096066227), (2, 8.38525491562421...",0.0
6,6,"[(1, 11.858541225631424), (2, 8.38525491562421...",0.0
7,7,"[(1, 11.858541225631424), (2, 11.8585412256314...",0.0
8,8,"[(1, 5.303300858899107), (2, 11.85854122563142...",0.0
9,9,"[(1, 5.303300858899107), (2, 7.5), (3, 0.0), (...",0.0


In [124]:
df['min_speed'] = df['speeds'].apply(lambda speeds: min([speed for _, speed in speeds]) if speeds else 0)
df['max_speed'] = df['speeds'].apply(lambda speeds: max([speed for _, speed in speeds]) if speeds else 0)
df['avg_speed'] = df['speeds'].apply(lambda speeds: np.mean([speed for _, speed in speeds]) if speeds else 0)

df[['frame', 'min_speed', 'max_speed', 'avg_speed']].head(15)


,frame,min_speed,max_speed,avg_speed
0,0,0.00,0.000000,0.000000
1,1,0.00,0.000000,0.000000
2,2,0.00,0.000000,0.000000
3,3,0.00,0.000000,0.000000
4,4,0.00,16.770510,6.292227
5,5,0.00,22.500000,9.064400
6,6,0.00,45.155980,11.494067
7,7,0.00,42.756578,10.049017
8,8,3.75,39.151149,11.546061
9,9,0.00,20.194368,5.890257


In [125]:
print(f"MIN_SPEED: min({df['min_speed'].min()}), max({df['min_speed'].max()})")

MIN_SPEED: min(0.0), max(1326.0638417889238)


In [126]:
print(f"MAX_SPEED: min({df['max_speed'].min()}), max({df['max_speed'].max()})")

MAX_SPEED: min(0.0), max(1947.4490526070254)


In [127]:
print(f"AVG_SPEED: min({df['avg_speed'].min()}), max({df['avg_speed'].max()})")

AVG_SPEED: min(0.0), max(1326.0638417889238)


In [128]:
df.head(100)

,frame,shepherd,frame_bboxes,min_distance,max_distance,avg_distance,furthest_sheep_distance,speeds,dog_speed,min_speed,max_speed,avg_speed
0,0,robot,"[(1, sheep, [260, 628, 638, 862]), (2, sheep, ...",NaN,NaN,NaN,1034.203558,"[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",0.0,0.0,0.000000,0.000000
1,1,robot,"[(1, sheep, [260, 629, 638, 862]), (2, sheep, ...",NaN,NaN,NaN,1033.859758,"[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",0.0,0.0,0.000000,0.000000
2,2,robot,"[(1, sheep, [260, 629, 637, 862]), (2, sheep, ...",NaN,NaN,NaN,1033.218515,"[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",0.0,0.0,0.000000,0.000000
3,3,robot,"[(1, sheep, [259, 629, 637, 862]), (2, sheep, ...",NaN,NaN,NaN,1034.157870,"[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0...",0.0,0.0,0.000000,0.000000
4,4,robot,"[(1, sheep, [259, 629, 636, 861]), (2, sheep, ...",NaN,NaN,NaN,1034.329372,"[(1, 11.25), (2, 0.0), (3, 5.303300858899107),...",0.0,0.0,16.770510,6.292227
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,robot,"[(1, sheep, [257, 631, 635, 857]), (2, sheep, ...",NaN,NaN,NaN,1030.870627,"[(1, 11.25), (2, 3.75), (3, 15.461646096066227...",0.0,0.0,26.250000,8.448217
96,96,robot,"[(1, sheep, [257, 632, 635, 857]), (2, sheep, ...",NaN,NaN,NaN,1031.809697,"[(1, 5.303300858899107), (2, 3.75), (3, 11.858...",0.0,0.0,30.233467,7.737865
97,97,robot,"[(1, sheep, [256, 632, 634, 857]), (2, sheep, ...",NaN,NaN,NaN,1031.809697,"[(1, 8.385254915624213), (2, 0.0), (3, 8.38525...",0.0,0.0,15.000000,5.457138
98,98,robot,"[(1, sheep, [256, 632, 634, 857]), (2, sheep, ...",NaN,NaN,NaN,1030.401135,"[(1, 8.385254915624213), (2, 3.75), (3, 7.5), ...",0.0,0.0,8.385255,5.221314


In [129]:
df.drop(columns=['frame_bboxes', 'speeds'], inplace=True)

In [130]:
df.to_csv(f'data/processed_data/{csv_name}.csv', index=False)